In [1]:
!pip install dill
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 4.2 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

In [2]:
import numpy as np
import dill
import timeit
import shap

In [ ]:
import matplotlib.pyplot as plt

from sklearn import linear_model

from tsmule.xai.lime import LimeTS
from tsmule.sampling.segment import WindowSegmentation, MatrixProfileSegmentation, SAXSegmentation
from tsmule.sampling.perturb import Perturbation
from tsmule.xai.evaluation import PerturbationAnalysis

In [6]:
#rnn_model = tf.keras.models.load_model('/content/drive/MyDrive/Nadiyah/final_codes/Beijing_Air_Quality_models/beijing_air_2_5_rnn_model.h5')

with open('/content/drive/MyDrive/Nadiyah/final_codes/Beijing_Air_Quality_models/beijing_air_2_5_test_data.dill', 'rb') as f:
    dataset_test = dill.load(f)

In [ ]:
sample=dataset_test[0] #taking 10,080 samples
labels=dataset_test[1]

In [ ]:
def predict_fn(x):
    if len(x.shape) == 2:
        prediction = rnn_model.predict(x[np.newaxis, :, :]).ravel()
    else:
        prediction = rnn_model.predict(x).ravel()
    return prediction

In [ ]:
tic=timeit.default_timer()
shap_exp=shap.DeepExplainer(rnn_model, sample[:3000]) #expected shape (None, 24, 7)

shap_values=shap_exp.shap_values(sample)

print('Total time: ' + str(timeit.default_timer()-tic))

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


Total time: 4624.5481931760005


In [ ]:
shap_values=np.asarray(shap_values).squeeze()
shap_values.shape

(8347, 24, 7)

In [ ]:
with open('beijing_air_rnn_shap_values.dill', 'wb') as f:
    dill.dump(shap_values, f)

In [ ]:
lasso_classifier = linear_model.Lasso(alpha=0.01)  #faster the model, faster LIME works
per=Perturbation() #perturbation object for LimeTS object, sampler

Explanation By Uniform Window Segmentation

In [ ]:
#LimeTS object for uniform segmentation
tic = timeit.default_timer()

#segments object, WindowSegmentation object has stationery and exponential segmentations techniques
uniform_seg=WindowSegmentation(partitions=4, win_length=24)
uniform_lime=LimeTS(kernel=lasso_classifier, segmenter=uniform_seg, sampler=per, n_samples=24)
lime_values_uni=[uniform_lime.explain(sample[i], predict_fn, segmentation_method='uniform')
                 for i in range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

1/1 [==============================] - 0s 270ms/step


In [ ]:
with open("beijing_air_rnn_lime_values_uni.dill", 'wb') as f:
    dill.dump(lime_values_uni, f)

(1, 24, 7)

Explanation for Exponential Window Segmentation

In [ ]:
#LimeTS object for exponential window segmentation
tic = timeit.default_timer()
#segment object, WindowSegmentation has stationery and exponentials segmentation techniques
exp_seg=WindowSegmentation(partitions=4, win_length=24)
exp_lime=LimeTS(kernel=lasso_classifier, segmenter=exp_seg, sampler=per, n_samples=24)
#explainer for LimeTS
lime_values_exp=[exp_lime.explain(sample[i], predict_fn, segmentation_method='exponential')
                 for i in range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

1/1 [==============================] - 0s 290ms/step


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.059e-04, tolerance: 1.063e-04


In [ ]:
with open("beijing_air_rnn_lime_values_exp.dill", 'wb') as f:
    dill.dump(lime_values_exp, f)

Explanation by LIME and Matrix Profile Segmentation

In [ ]:
#LimeTS object for matrix profile segmentation= slopes-sorted
tic=timeit.default_timer()
#segment object, MatrixProfileSegmentation object has slopes, bins-min, bins-max segmentation techniques
seg_slopes=MatrixProfileSegmentation(partitions=4, win_length=20)

lime_slopes=LimeTS(kernel=lasso_classifier, segmenter=seg_slopes, sampler=per)
lime_values_slopes=[lime_slopes.explain(sample[i], predict_fn,
                                                  segmentation_method='slopes-sorted') for i in
                    range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

A large number of values are smaller than 1e-05.
For a self-join, try setting `ignore_trivial = True`.


1/1 [==============================] - 0s 137ms/step


In [ ]:
with open("beijing_air_rnn_lime_values_slopes.dill", 'wb') as f:
    dill.dump(lime_values_slopes, f)

Explanation with LIME and SAX Segmentation

In [ ]:
#LimeTS object for SAX segmentation
tic=timeit.default_timer()
#create segment object for SAX Transformation
seg_sax=SAXSegmentation(partitions=4, win_length=10)

lime_sax=LimeTS(kernel=lasso_classifier, segmenter=seg_sax, sampler=per, n_samples=24)
lime_values_sax=[lime_sax.explain(sample[i], predict_fn) for i in range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.


1/1 [==============================] - 0s 144ms/step


In [ ]:
with open("beijing_air_rnn_lime_values_sax.dill", 'wb') as f:
    dill.dump(lime_values_sax, f)